In [371]:
import sagemaker
from sagemaker.pytorch import PyTorchModel
from sagemaker import get_execution_role
import boto3
import os
import tarfile
import numpy as np
from PIL import Image
import json
from torchvision import transforms
import torch


In [372]:
sagemaker_session = sagemaker.Session()
role = get_execution_role()

In [378]:
# Create tar.gz file w/ the model and inference script
def create_model_tarball():
    with tarfile.open('model.tar.gz', 'w:gz') as tar:
        tar.add('best_model_2.pth')
        tar.add('code/inference.py')

create_model_tarball()

In [379]:
# Upload the model.tar.gz to S3
model_data = sagemaker_session.upload_data(
    'model.tar.gz',
    key_prefix='pathmnist-model'
)

In [380]:
# Create PyTorch model
pytorch_model = PyTorchModel(
    model_data=model_data,
    role=role,
    entry_point='code/inference.py',
    framework_version='2.3.0',
    py_version='py311',
    predictor_cls=sagemaker.pytorch.model.PyTorchPredictor
)

In [381]:
# Deploy model
predictor = pytorch_model.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.medium',
    endpoint_name='pathmnist-endpoint',
    client_config={'request_timeout': 120}
)

[01/09/25 22:59:35] INFO     Repacking model artifact                                                  ]8;id=990939;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/model.py\model.py]8;;\:]8;id=760103;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/model.py#819\819]8;;\
                             (s3://sagemaker-us-east-2-184438101811/pathmnist-model/model.tar.gz),                 
                             script artifact (None), and dependencies ([]) into single tar.gz file                 
                             located at                                                                            
                             s3://sagemaker-us-east-2-184438101811/pytorch-inference-2025-01-09-22-59-             
                             35-424/model.tar.gz. This may take some time depending on model size...               

[01/09/25 22:59:36] INFO     Creating model with name: pytorch-inference-2025-01-09-22-59-36-012    ]8;id=106023;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=639270;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

                    INFO     Creating endpoint-config with name pathmnist-endpoint                  ]8;id=2202;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=522806;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#5889\5889]8;;\

                    INFO     Creating endpoint with name pathmnist-endpoint                         ]8;id=295366;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=39299;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4711\4711]8;;\

------------!

In [382]:
sagemaker_client = boto3.client('sagemaker')

response = sagemaker_client.describe_endpoint(EndpointName='pathmnist-endpoint')
print("Endpoint Status:", response['EndpointStatus'])

Endpoint Status: InService


In [383]:
def predict_single_image(image_path, endpoint_name):
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])
    
    # Load and preprocess image
    image = Image.open(image_path).convert('RGB')
    
    image_tensor = transform(image)
    image_array = image_tensor.numpy()
    
    print(f"Image shape: {image_array.shape}") # expect (3, 28, 28)
    print(f"Value range: [{image_array.min():.3f}, {image_array.max():.3f}]")
    
    runtime = boto3.client('sagemaker-runtime')
    response = runtime.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType='application/json',
        Body=json.dumps(image_array.tolist())
    )
    
    prediction = json.loads(response['Body'].read().decode())
    return prediction



In [394]:
prediction = predict_single_image('sample_6869_label_[4].png', 'pathmnist-endpoint')
prediction

Image shape: (3, 28, 28)
Value range: [-1.143, 2.135]


[4]

In [377]:
# Clean up - delete the endpoint when done
predictor.delete_endpoint()

[01/09/25 22:59:26] INFO     Deleting endpoint configuration with name: pathmnist-endpoint          ]8;id=580804;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=65979;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4865\4865]8;;\

                    INFO     Deleting endpoint with name: pathmnist-endpoint                        ]8;id=453196;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=367877;file:///home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/sagemaker/session.py#4855\4855]8;;\